In [1]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

doc2start = {"bpt6k63243601": 123, "bpt6k62931221": 151, "bpt6k6286466w": 189, "bpt6k6393838j": 219, "bpt6k6331310g": 216, "bpt6k6292987t": 353, "bpt6k62906378": 288, "bpt6k6391515w": 319, "bpt6k6315927h": 349, "bpt6k6319106t": 324, "bpt6k6315985z": 82, "bpt6k63959929": 82, "bpt6k63197984": 56, "bpt6k6389871r": 77, "bpt6k6319811j": 79, "bpt6k6282019m": 72, "bpt6k6314752k": 190, "bpt6k6305463c": 113, "bpt6k6318531z": 108, "bpt6k6324389h": 72, "bpt6k63243920": 80, "bpt6k6309075f": 96, "bpt6k6333200c": 132, "bpt6k63243905": 134, "bpt6k6333170p": 137, "bpt6k96727875": 135, "bpt6k9764746t": 99, "bpt6k97645375": 123, "bpt6k9672117f": 125, "bpt6k9763554c": 123, "bpt6k9763553z": 105, "bpt6k9677392n": 110, "bpt6k9692809v": 113, "bpt6k9762929c": 129, "bpt6k9672776c": 119, "bpt6k9764647w": 121, "bpt6k9669143t": 145, "bpt6k9677737t": 139, "bpt6k9668037f": 167, "bpt6k96839542": 171, "bpt6k96762564": 185, "bpt6k9685861g": 189, "bpt6k9763471j": 153, "bpt6k9762899p": 157, "bpt6k97630871": 11, "bpt6k9684454n": 235, "bpt6k9732740w": 239, "bpt6k9684013b": 189, "bpt6k9692626p": 305, "bpt6k9685098r": 281, "bpt6k9764402m": 329, "bpt6k97631451": 322, "bpt6k9776121t": 49, "bpt6k9775724t": 33, "bpt6k97774838": 327, "bpt6k9780089g": 339}

def entry2url(row):
    """
    Takes a row of an Annuaire csv and
    transforms it to the corresponding Gallica url
    """
    url = "https://gallica.bnf.fr/ark:/12148/"
    
    directory = row['directory']
    page = row['page'] - doc2start[directory]
    url += f"{row['directory']}/f{row['page']-doc2start[row['directory']]}"
    
    r_strings = []
    if 'name' in row and pd.notna(row['name']):
        r_strings.append(quote(row['name'].replace('.', ' ')))
    if 'job' in row and pd.notna(row['job']):
        r_strings.append(quote(row['job'].replace('.', ' ')))
    if 'street' in row and pd.notna(row['street']):
        r_strings.append(quote(row['street'].replace('.', ' ')))
    if 'number' in row and pd.notna(row['number']):
        r_strings.append(quote(row['number'].replace('.', ' ')))
    
    if len(r_strings) > 0:
        url += f".item.r={'%20'.join(r_strings)}.zoom"
    
    return url

def add_clickable_url(bottin_dataframe):
    bottin_dataframe = bottin_dataframe.copy()
    bottin_dataframe['url'] = bottin_dataframe.apply(entry2url, axis=1)
    #def make_clickable(val):
    #    return '<a href="{}">gallica url</a>'.format(val,val)

    #return bottin_dataframe.style.format(make_clickable, subset=['url'])
    return bottin_dataframe

In [2]:
data_bottin = pd.read_csv('bottin_data_groupe_6.csv')

In [3]:
#nettoyer le numéro de rue: garder seulement le premier nombre et éventuellement 'bis'
data_bottin['number_clean'] = data_bottin['number'].str.extract('(^\d+(?: ?bis)?).*')

In [4]:
data_bottin = add_clickable_url(data_bottin.fillna(''))

In [5]:
#Generate text for wikipast pages
df = pd.DataFrame()

In [6]:
df['criteria'] = data_bottin['name'].astype(str)+" "\
+data_bottin['job'].astype(str)+" "\
+data_bottin['number_clean'].astype(str)+" "\
+data_bottin['street_clean'].astype(str)

In [7]:
df['page_text'] = "[["+data_bottin['year'].astype(str)+"]]"\
+" / [[Paris]]. "\
+"[["+data_bottin['name'].astype(str)+"]]"\
+", "+ data_bottin['job'].astype(str)\
+", exerce son activité au "\
+"[["+data_bottin['street_clean'].astype(str)+"]]"\
+" "+data_bottin['number_clean'].astype(str)+"."\
+" ["+data_bottin['url'].astype(str)+"]"

In [8]:
pd.set_option('display.max_colwidth', None)
df.head(1)

,criteria,page_text
0,Aaron march. de chevaux 39 rue Bouret,"[[1896]] / [[Paris]]. [[Aaron]], march. de chevaux, exerce son activité au [[rue Bouret]] 39. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Aaron%20march%20%20de%20chevaux%20r%20%20Bouret%2039.zoom]"
